Preprocessing Anime dataset from Kaggle

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt

Data preprocessing: cleaning initial dataset to remove unwanted entries and columns

In [11]:
anime_df = pd.read_csv("anime.csv")
anime_df.drop(columns=["members"], axis = 1, inplace = True)
anime_df.head(10)

# remove entries where user has not rated an anime (indicated by a -1 in the rating column)
user_ratings_df = pd.read_csv("rating.csv")
user_ratings_df = user_ratings_df[user_ratings_df.rating != -1]
user_ratings_df.head(10)


7813737
6337241


,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
156,3,20,8
157,3,154,6
158,3,170,9
159,3,199,10
160,3,225,9
